In [2]:
import torch
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        
        self.hidden_size = hidden_size
        
        self.rnn = nn.RNN(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        h0 = torch.zeros(1, x.size(1), self.hidden_size)
        
        out, hn = self.rnn(x, h0)
        
        out = self.fc(hn[-1])
        out = torch.sigmoid(out)
        
        return out


In [17]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self):
        self.data = [
            (
                torch.tensor([[-1, 0, 1], [1, -1, 0], [0, 1, -1]], dtype=torch.float),
                torch.tensor([1], dtype=torch.float)
            ),
            (
                torch.tensor([[1, 0, -1], [-1, 1, 0], [0, -1, 1]], dtype=torch.float),
                torch.tensor([0], dtype=torch.float)
            ),
            # add more samples as needed
        ]
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]


In [18]:
batch_size = 1
dataset = CustomDataset()
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [19]:
input_size = 3
hidden_size = 10
output_size = 1

rnn = RNN(input_size, hidden_size, output_size)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.01)


In [21]:
num_epochs = 100

for epoch in range(num_epochs):
    total_loss = 0
    
    for inputs, targets in dataloader:
        rnn.zero_grad()
        
        outputs = rnn(inputs.permute(1, 0, 2))
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, total_loss))


Epoch [1/100], Loss: 1.4043
Epoch [2/100], Loss: 1.2458
Epoch [3/100], Loss: 1.1163
Epoch [4/100], Loss: 0.9936
Epoch [5/100], Loss: 0.8766
Epoch [6/100], Loss: 0.7651
Epoch [7/100], Loss: 0.6616
Epoch [8/100], Loss: 0.5630
Epoch [9/100], Loss: 0.4773
Epoch [10/100], Loss: 0.4002
Epoch [11/100], Loss: 0.3358
Epoch [12/100], Loss: 0.2822
Epoch [13/100], Loss: 0.2381
Epoch [14/100], Loss: 0.2031
Epoch [15/100], Loss: 0.1737
Epoch [16/100], Loss: 0.1496
Epoch [17/100], Loss: 0.1293
Epoch [18/100], Loss: 0.1134
Epoch [19/100], Loss: 0.1002
Epoch [20/100], Loss: 0.0898
Epoch [21/100], Loss: 0.0801
Epoch [22/100], Loss: 0.0729
Epoch [23/100], Loss: 0.0660
Epoch [24/100], Loss: 0.0605
Epoch [25/100], Loss: 0.0557
Epoch [26/100], Loss: 0.0517
Epoch [27/100], Loss: 0.0481
Epoch [28/100], Loss: 0.0452
Epoch [29/100], Loss: 0.0422
Epoch [30/100], Loss: 0.0399
Epoch [31/100], Loss: 0.0375
Epoch [32/100], Loss: 0.0356
Epoch [33/100], Loss: 0.0338
Epoch [34/100], Loss: 0.0322
Epoch [35/100], Loss: 0

In [1]:
import numpy as np 
p = 0.5
measurement_record_0 = np.load("measurement_record_p_{}_Q_{}.npy".format(p,0))
measurement_record_1 = np.load("measurement_record_p_{}_Q_{}.npy".format(p,1))
measurement_records = np.concatenate([measurement_record_0,measurement_record_1],axis=0)
depth = len(measurement_record_0[0,:,0]) # excluding very last layer containing final measurements
L = len(measurement_record_0[0,0,:])
num_meas_records_0 = len(measurement_record_0[:,0,0])
num_meas_records_1 = len(measurement_record_1[:,0,0])
num_meas_records = num_meas_records_0+num_meas_records_1
charge_output_0 = np.zeros(num_meas_records_0)
charge_output_1 = np.ones(num_meas_records_1)
charge_output = np.concatenate([charge_output_0,charge_output_1],axis=0)
p = np.random.permutation(num_meas_records) 
X = measurement_records[p,:,:]
y = charge_output[p]

In [3]:
rnn = RNN(np.shape(measurement_record_0[0,:,:]), depth*L, 2)

TypeError: empty(): argument 'size' must be tuple of SymInts, but found element of type tuple at pos 2

In [16]:
import tensorflow.compat.v1 as tf
import numpy as np

## Disable eager execution
#tf.disable_eager_execution()

# Define the input shape
input_shape = [10, 10]

# Define the input placeholder
inputs = tf.keras.layers.Input(shape=input_shape, name='inputs')

# Define the RNN cell
cell = tf.keras.layers.SimpleRNN(units=64)

# Define the RNN layer
outputs = cell(inputs)

# Define the fully connected layer
dense = tf.keras.layers.Dense(units=1, activation=tf.nn.sigmoid)(outputs)

# Define the output
output = tf.squeeze(tf.round(dense))

# Define the model
model = tf.keras.Model(inputs=inputs, outputs=output)

# Compile the model
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=[tf.keras.metrics.BinaryAccuracy()])

# Generate some dummy data
data = np.random.randint(-1, 2, size=[100, 10, 10])
labels = np.random.randint(0, 2, size=[100, 1])

# Train the model
model.fit(data, labels, epochs=10, batch_size=32)

# Test the model on some new data
test_data = np.random.randint(-1, 2, size=[1, 10, 10])
result = model.predict(test_data)
print("Result:", result)


Epoch 1/10


2023-03-15 16:43:48.796338: W tensorflow/c/c_api.cc:291] Operation '{name:'total_5/Assign' id:1091 op device:{requested: '', assigned: ''} def:{{{node total_5/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](total_5, total_5/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


AssertionError: 